In [47]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Classificaton

In [48]:
df = pd.read_excel('Bike Sales Dashboard.xlsx')
df.head(5)

,ID,Martial Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Age Brackets,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,Middle Age,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,Middle Age,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,Old,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Middle Age,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Middle Age,Yes


In [49]:
df.columns = df.columns.str.replace(' ', '')
df.columns

Index(['ID', 'MartialStatus', 'Gender', 'Income', 'Children', 'Education',
       'Occupation', 'HomeOwner', 'Cars', 'CommuteDistance', 'Region', 'Age',
       'AgeBrackets', 'PurchasedBike'],
      dtype='object')

In [50]:
df.drop(['ID'], axis = 1, inplace = True)
df

,MartialStatus,Gender,Income,Children,Education,Occupation,HomeOwner,Cars,CommuteDistance,Region,Age,AgeBrackets,PurchasedBike
0,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,Middle Age,No
1,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,Middle Age,No
2,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,Old,No
3,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Middle Age,Yes
4,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Middle Age,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Married,Male,60000,2,High School,Professional,Yes,2,2-5 Miles,North America,54,Middle Age,Yes
996,Single,Male,70000,4,Graduate Degree,Professional,Yes,0,2-5 Miles,North America,35,Middle Age,Yes
997,Married,Male,60000,2,Bachelors,Skilled Manual,Yes,0,0-1 Miles,North America,38,Middle Age,Yes
998,Single,Male,100000,3,Bachelors,Management,No,3,1-2 Miles,North America,38,Middle Age,No


In [51]:
df = pd.get_dummies(df, columns = ['Occupation', 'Region'])

In [52]:
label_encode = LabelEncoder()
df['MartialStatus'] = label_encode.fit_transform(df['MartialStatus'])
df['Gender'] = label_encode.fit_transform(df['Gender'])
df['HomeOwner'] = label_encode.fit_transform(df['HomeOwner'])
df['PurchasedBike'] = label_encode.fit_transform(df['PurchasedBike'])

In [53]:
education = {'Partial High School': 0, 'High School': 1, 'Partial College': 2, 'Bachelors': 3, 'Graduate Degree': 4}
df['Education'] = df['Education'].map(education)

In [54]:
age_categories = {'Adolescent': 0, 'Middle Age': 1, 'Old': 3}
df['AgeBrackets'] = df['AgeBrackets'].map(age_categories)

In [55]:
distance = {'0-1 Miles': 0, '1-2 Miles': 1, '2-5 Miles': 2, '5-10 Miles': 3, 'More than 10 Miles': 4}
df['CommuteDistance'] = df['CommuteDistance'].map(distance)

In [56]:
df

,MartialStatus,Gender,Income,Children,Education,HomeOwner,Cars,CommuteDistance,Age,AgeBrackets,PurchasedBike,Occupation_Clerical,Occupation_Management,Occupation_Manual,Occupation_Professional,Occupation_Skilled Manual,Region_Europe,Region_North America,Region_Pacific
0,0,0,40000,1,3,1,0,0,42,1,0,0,0,0,0,1,1,0,0
1,0,1,30000,3,2,1,1,0,43,1,0,1,0,0,0,0,1,0,0
2,0,1,80000,5,2,0,2,2,60,3,0,0,0,0,1,0,1,0,0
3,1,1,70000,0,3,1,1,3,41,1,1,0,0,0,1,0,0,0,1
4,1,1,30000,0,3,0,0,0,36,1,1,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,1,60000,2,1,1,2,2,54,1,1,0,0,0,1,0,0,1,0
996,1,1,70000,4,4,1,0,2,35,1,1,0,0,0,1,0,0,1,0
997,0,1,60000,2,3,1,0,0,38,1,1,0,0,0,0,1,0,1,0
998,1,1,100000,3,3,0,3,1,38,1,0,0,1,0,0,0,0,1,0


In [57]:
X = df.drop(['PurchasedBike'], axis=1)
y = df['PurchasedBike']

In [58]:
xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=0.2, random_state=42)

In [87]:
xtrain.shape

(800, 18)

In [61]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(18,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                1216      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 3841 (15.00 KB)
Trainable params: 3841 (15.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [62]:
model.fit(xtrain, ytrain, epochs=10, batch_size=32)

Epoch 1/10
25/25 [==============================] - 1s 3ms/step - loss: 521.7125 - accuracy: 0.4837
Epoch 2/10
25/25 [==============================] - 0s 3ms/step - loss: 134.7437 - accuracy: 0.4837
Epoch 3/10
25/25 [==============================] - 0s 3ms/step - loss: 42.4038 - accuracy: 0.4837
Epoch 4/10
25/25 [==============================] - 0s 3ms/step - loss: 50.7222 - accuracy: 0.4837
Epoch 5/10
25/25 [==============================] - 0s 3ms/step - loss: 130.1275 - accuracy: 0.4837
Epoch 6/10
25/25 [==============================] - 0s 2ms/step - loss: 64.1659 - accuracy: 0.4837
Epoch 7/10
25/25 [==============================] - 0s 3ms/step - loss: 118.6467 - accuracy: 0.4837
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: 181.1572 - accuracy: 0.4837
Epoch 9/10
25/25 [==============================] - 0s 4ms/step - loss: 33.3893 - accuracy: 0.4837
Epoch 10/10
25/25 [==============================] - 0s 4ms/step - loss: 75.9377 - accuracy: 0.4837


In [63]:
_, ac = model.evaluate(xtest, ytest)
print(ac)

7/7 [==============================] - 0s 3ms/step - loss: 42.8135 - accuracy: 0.4700
0.4699999988079071


# Regression

In [90]:
df1 = pd.read_csv('dress_rental_prices.csv')
df1

,Unnamed: 0,ID,Name,Brand,Colour,Catagories,Price
0,0,74416,Runway stripe dress,Stella McCartney,beige,dresses,111
1,1,73815,Reformation Kourtney Dress,Reformation,beige,dresses,50
2,2,73801,Ivory Viola bridal dress,Ghost,beige,dresses,75
3,3,73718,Pasu Dress - Rhino Tusk,Coucoo,beige,dresses,37
4,4,73605,Ellen,RIXO,beige,dresses,47
...,...,...,...,...,...,...,...
29992,29992,872,Silk and wool-blend crepe dress yellow,Victoria Beckham,yellow,"dresses, Smart, midi, Short Sleeve, Pencil, Mo...",102
29993,29993,871,La Guardia Matching Set,Faithfull The Brand,yellow,"dresses, Floral, Polka Dot, Loose, Smart Casua...",43
29994,29994,844,One-shoulder ruffled crepe mini dress,Solace London,yellow,"dresses, Vinyl-PVC-Synthetics, Smart, Formal, ...",75
29995,29995,592,Billie Dress,Rebecca Vallance,yellow,"dresses, Cotton-Jersey, Tie-Front, Skinny-Fit,...",42


In [91]:
df1 = df1.drop(['Unnamed: 0'],axis=1)
df1

,ID,Name,Brand,Colour,Catagories,Price
0,74416,Runway stripe dress,Stella McCartney,beige,dresses,111
1,73815,Reformation Kourtney Dress,Reformation,beige,dresses,50
2,73801,Ivory Viola bridal dress,Ghost,beige,dresses,75
3,73718,Pasu Dress - Rhino Tusk,Coucoo,beige,dresses,37
4,73605,Ellen,RIXO,beige,dresses,47
...,...,...,...,...,...,...
29992,872,Silk and wool-blend crepe dress yellow,Victoria Beckham,yellow,"dresses, Smart, midi, Short Sleeve, Pencil, Mo...",102
29993,871,La Guardia Matching Set,Faithfull The Brand,yellow,"dresses, Floral, Polka Dot, Loose, Smart Casua...",43
29994,844,One-shoulder ruffled crepe mini dress,Solace London,yellow,"dresses, Vinyl-PVC-Synthetics, Smart, Formal, ...",75
29995,592,Billie Dress,Rebecca Vallance,yellow,"dresses, Cotton-Jersey, Tie-Front, Skinny-Fit,...",42


In [92]:
df1.drop(['ID'], axis = 1, inplace = True)

In [93]:
df1.isna().sum()

Name            2
Brand           0
Colour          0
Catagories    114
Price           0
dtype: int64

In [94]:
mode_value = df1["Name"].mode()[0]
mode_value1 = df1['Catagories'].mode()[0]

In [95]:
df1['Name'].fillna(mode_value, inplace=True)
df1['Catagories'].fillna(mode_value1, inplace=True)

In [97]:
df1.drop(['Name', 'Catagories'], axis = 1, inplace = True)

In [99]:
df1 = pd.get_dummies(df1, columns = ['Brand', 'Colour'])

In [100]:
df1.head(2)

,Price,Brand_& For Love,Brand_& Other Stories,Brand_1017 ALYX 9SM,Brand_12 storeez,Brand_16ARLINGTON,Brand_3.1 Phillip Lim,Brand_A Star is Born,Brand_A.L.C,Brand_A.P.C.,...,Colour_navy,Colour_orange,Colour_phantom,Colour_pink,Colour_purple,Colour_red,Colour_silver,Colour_tan,Colour_white,Colour_yellow
0,111,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,50,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
x = df1.drop(['Price'], axis = 1)
y = df1['Price']

In [102]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [103]:
X_train.shape

(23997, 951)

In [105]:
model = Sequential([
    Dense(units = 64, input_shape = (951, ), activation = 'relu'), 
    Dense(units = 32, activation = 'relu'),
    Dense(units = 16, activation = 'relu'),
    Dense(units = 1, activation = 'linear')
])
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

In [106]:
model.fit(X_train, y_train, epochs = 10, batch_size = 128)

Epoch 1/10
188/188 [==============================] - 3s 6ms/step - loss: 3041.5715 - mae: 39.7245
Epoch 2/10
188/188 [==============================] - 1s 6ms/step - loss: 1036.0439 - mae: 19.2163
Epoch 3/10
188/188 [==============================] - 1s 7ms/step - loss: 853.6315 - mae: 16.9225
Epoch 4/10
188/188 [==============================] - 1s 6ms/step - loss: 795.8784 - mae: 16.1451
Epoch 5/10
188/188 [==============================] - 1s 6ms/step - loss: 769.3450 - mae: 15.8125
Epoch 6/10
188/188 [==============================] - 1s 6ms/step - loss: 753.6954 - mae: 15.6788
Epoch 7/10
188/188 [==============================] - 1s 6ms/step - loss: 743.0977 - mae: 15.5265
Epoch 8/10
188/188 [==============================] - 1s 5ms/step - loss: 735.3531 - mae: 15.4523
Epoch 9/10
188/188 [==============================] - 1s 5ms/step - loss: 728.8373 - mae: 15.3955
Epoch 10/10
188/188 [==============================] - 1s 5ms/step - loss: 724.1388 - mae: 15.3404


In [108]:
loss, mae = model.evaluate(X_test, y_test)
print(f'MAE: {mae}')

188/188 [==============================] - 1s 4ms/step - loss: 808.1111 - mae: 15.9530
MAE: 15.953015327453613
